In [1]:
# ==========================================
# 1. INSTALAÇÃO (Colab)
# ==========================================
!pip install -U -q google-search-results langchain-google-genai langchain langchain-community

In [7]:
# ==========================================
# 2. CONFIGURAÇÃO E IMPORTS
# ==========================================
import os
import getpass
import json
from langchain_google_genai import ChatGoogleGenerativeAI
from serpapi import GoogleSearch
from datetime import datetime, timedelta

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Cole sua Google AI Key: ")

if "SERPAPI_API_KEY" not in os.environ:
    os.environ["SERPAPI_API_KEY"] = getpass.getpass("Cole sua SerpAPI Key: ")

SERPAPI_KEY = os.environ["SERPAPI_API_KEY"]

# ==========================================
# PRETTY PRINT – saída estruturada da API
# ==========================================
def print_estruturado(titulo, dados):
    print(f"\n🟦 {titulo}\n")
    try:
        print(json.dumps(dados, indent=4, ensure_ascii=False))
    except:
        print(dados)


# ==========================================
# 3. GERAR DATAS + INTERESSES VIA LLM
# ==========================================
def gerar_dados_viagem(origem, destino):
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

    prompt = f"""
    Escolha datas ideais para uma viagem de {origem} até {destino}.
    Regras:
    - viagem deve ter entre 5 e 12 dias.
    - escolha um mês em alta ou média temporada.
    - interesses devem ser variados (cultura, gastronomia, natureza etc.)

    Responda no formato:
    IDA: AAAA-MM-DD
    VOLTA: AAAA-MM-DD
    INTERESSES: lista separada por vírgula
    """

    resposta = llm.invoke(prompt).content
    linhas = resposta.split("\n")

    data_ida = linhas[0].split(":")[1].strip()
    data_volta = linhas[1].split(":")[1].strip()
    interesses = linhas[2].split(":")[1].strip()

    return data_ida, data_volta, interesses


# ==========================================
# 4. TRADUZIR CIDADE -> IATA
# ==========================================
def descobrir_aeroporto(cidade):
    print(f"🤔 Identificando aeroporto principal de: {cidade}...")

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

    prompt = f"""
    Qual é o código IATA (3 letras) do principal aeroporto internacional de: {cidade}?
    Responda apenas a sigla.
    """

    codigo = llm.invoke(prompt).content.strip()
    print(f"   ✅ Código encontrado: {codigo}")
    return codigo


# ==========================================
# 5. SERPAPI — BUSCA DE VOOS
# ==========================================
def buscar_voos_reais(origem, destino_iata, data_ida, data_volta):
    print(f"✈️  Buscando voos ({origem} -> {destino_iata})...")

    try:
        params = {
            "engine": "google_flights",
            "departure_id": origem,
            "arrival_id": destino_iata,
            "outbound_date": data_ida,
            "return_date": data_volta,
            "currency": "BRL",
            "hl": "pt",
            "api_key": SERPAPI_KEY
        }

        results = GoogleSearch(params).get_dict()
        print_estruturado("RESULTADO SERPAPI (VOOS)", results)

        voos = results.get("best_flights", []) or results.get("other_flights", [])
        if not voos:
            return "Nenhum voo encontrado."

        lista = []
        for v in voos[:3]:
            preco = v.get("price", "N/A")
            duracao = v["flights"][0].get("duration", "N/A")
            cia = v["flights"][0].get("airline", "Companhia")
            lista.append(f"- {cia}: R$ {preco} ({duracao} min)")

        return "\n".join(lista)

    except Exception as e:
        return f"Erro voos: {e}"


# ==========================================
# 6. SERPAPI — HOTÉIS
# ==========================================
def buscar_hoteis_reais(destino, checkin, checkout):
    print(f"🏨  Buscando hotéis em {destino}...")

    try:
        params = {
            "engine": "google_hotels",
            "q": f"hotéis em {destino}",
            "check_in_date": checkin,
            "check_out_date": checkout,
            "currency": "BRL",
            "hl": "pt",
            "api_key": SERPAPI_KEY
        }

        results = GoogleSearch(params).get_dict()
        print_estruturado("RESULTADO SERPAPI (HOTÉIS)", results)

        props = results.get("properties", [])[:3]
        if not props:
            return "Nenhum hotel encontrado."

        lista = []
        for h in props:
            nome = h.get("name", "Hotel")
            preco = h.get("rate_per_night", {}).get("lowest", "N/A")
            nota = h.get("overall_rating", "S/N")
            lista.append(f"- {nome}: {preco} (Nota: {nota})")

        return "\n".join(lista)

    except Exception as e:
        return f"Erro hotéis: {e}"


# ==========================================
# 7. SERPAPI — ATRAÇÕES
# ==========================================
def pesquisar_atracoes(destino, interesses):
    print(f"🔍  Pesquisando atrações em {destino}...")

    try:
        params = {
            "engine": "google",
            "q": f"melhores atrações {destino} {interesses}",
            "api_key": SERPAPI_KEY,
            "hl": "pt",
            "gl": "br"
        }

        results = GoogleSearch(params).get_dict()
        print_estruturado("RESULTADO SERPAPI (ATRAÇÕES)", results)

        org = results.get("organic_results", [])
        if not org:
            return "Nenhuma atração encontrada."

        return "\n".join(
            f"- {i.get('title')}: {i.get('snippet', '')}"
            for i in org[:4]
        )

    except:
        return "Erro atrações."


# ==========================================
# 8. SERPAPI — CÂMBIO
# ==========================================
def buscar_cambio(pais):
    print(f"💱 Buscando câmbio do país: {pais} ...")

    try:
        params = {
            "engine": "google",
            "q": f"cotação moeda {pais} hoje",
            "api_key": SERPAPI_KEY,
            "hl": "pt",
            "gl": "br"
        }

        results = GoogleSearch(params).get_dict()
        print_estruturado("RESULTADO SERPAPI (CÂMBIO)", results)

        org = results.get("organic_results", [])
        if not org:
            return "Câmbio não encontrado."

        return org[0].get("snippet", "Sem informações.")

    except Exception as e:
        return f"Erro câmbio: {e}"


# ==========================================
# 9. SERPAPI — HISTÓRIA
# ==========================================
def pesquisar_historia(destino):
    print(f"📚 Pesquisando história de {destino}...")

    try:
        params = {
            "engine": "google",
            "q": f"história de {destino}",
            "api_key": SERPAPI_KEY,
            "hl": "pt",
            "gl": "br"
        }

        results = GoogleSearch(params).get_dict()
        print_estruturado("RESULTADO SERPAPI (HISTÓRIA)", results)

        org = results.get("organic_results", [])
        if not org:
            return "História não encontrada."

        return "\n".join(
            f"- {item.get('title')}: {item.get('snippet', '')}"
            for item in org[:3]
        )

    except:
        return "Erro história."


# ==========================================
# 10. ORQUESTRADOR COMPLETO
# ==========================================
def agente_de_viagens(origem_usuario, destino_usuario):
    # 1. LLM escolhe datas e interesses
    data_ida, data_volta, interesses = gerar_dados_viagem(origem_usuario, destino_usuario)

    # 2. Descobrir aeroportos
    codigo_origem = descobrir_aeroporto(origem_usuario)
    codigo_destino = descobrir_aeroporto(destino_usuario)

    # 3. Consultar dados reais
    voos = buscar_voos_reais(codigo_origem, codigo_destino, data_ida, data_volta)
    hoteis = buscar_hoteis_reais(destino_usuario, data_ida, data_volta)
    atracoes = pesquisar_atracoes(destino_usuario, interesses)
    cambio = buscar_cambio(destino_usuario)
    historia = pesquisar_historia(destino_usuario)

    # 4. Gerar roteiro final
    print("\n🤖  Gerando roteiro final...\n")
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

    prompt = f"""
    Gere um roteiro extremamente detalhado para uma viagem de {origem_usuario} para {destino_usuario}.

    DADOS REAIS:
    ✈️ Voos:
    {voos}

    🏨 Hotéis:
    {hoteis}

    🎡 Atrações:
    {atracoes}

    💱 Câmbio:
    {cambio}

    📚 História:
    {historia}

    DATAS:
    - Ida: {data_ida}
    - Volta: {data_volta}

    Interesses: {interesses}

    Monte:
    - Roteiro dia a dia baseado nos dados reais
    - Explicações culturais e históricas
    - Custos aproximados
    - Justificativa da escolha do roteiro
    """

    return llm.invoke(prompt).content


# ==========================================
# 11. TESTE FINAL
# ==========================================
ORIGEM = "São Paulo"
DESTINO = "Japão"

roteiro = agente_de_viagens(ORIGEM, DESTINO)
print("="*80)
print(roteiro)
print("="*80)


🤔 Identificando aeroporto principal de: São Paulo...
   ✅ Código encontrado: GRU
🤔 Identificando aeroporto principal de: Japão...
   ✅ Código encontrado: NRT
✈️  Buscando voos (GRU -> NRT)...

🟦 RESULTADO SERPAPI (VOOS)

{
    "error": "`outbound_date` cannot be in the past."
}
🏨  Buscando hotéis em Japão...

🟦 RESULTADO SERPAPI (HOTÉIS)

{
    "error": "`check_in_date` cannot be in the past."
}
🔍  Pesquisando atrações em Japão...

🟦 RESULTADO SERPAPI (ATRAÇÕES)

{
    "search_metadata": {
        "id": "6930af3f7223f4b81916d84e",
        "status": "Success",
        "json_endpoint": "https://serpapi.com/searches/cefed78b98f75002/6930af3f7223f4b81916d84e.json",
        "pixel_position_endpoint": "https://serpapi.com/searches/cefed78b98f75002/6930af3f7223f4b81916d84e.json_with_pixel_position",
        "created_at": "2025-12-03 21:44:31 UTC",
        "processed_at": "2025-12-03 21:44:31 UTC",
        "google_url": "https://www.google.com/search?q=melhores+atra%C3%A7%C3%B5es+Jap%C3%A3o+Cu